Create a conversion list for all rgi_ids to their terminus position, if terminus not available fallback to centroid coordinates.

In [19]:
from oggm import utils
import pandas as pd
import numpy as np
import os

# open centroid data as template structure

In [49]:
# open list of rgi_ids
frgi = utils.file_downloader('https://cluster.klima.uni-bremen.de/~oggm/rgi/rgi62_stats.h5')
df_rgi = pd.read_hdf(frgi, index_col=0)

# Template for final structure, only keep centroid coordinates
df_rgi = df_rgi[['CenLon', 'CenLat']]
df_rgi = df_rgi.rename(columns={'CenLon': 'centroid_lon', 'CenLat': 'centroid_lat'})

# Add terminus coordinates

In [67]:
df_rgi['terminus_lon'] = np.nan
df_rgi['terminus_lat'] = np.nan

terminus_fp = ('/home/www/oggm/gdirs/oggm_v1.6/L1-L2_files/elev_bands/RGI62/b_160/L2/summary/')

# add terminus region per region
for rgi_region in np.arange(1, 20):
    terminus_filename = f'glacier_statistics_{rgi_region:02d}.csv'
    df_terminus = pd.read_csv(os.path.join(terminus_fp, terminus_filename), index_col=0, low_memory=False)
    df_terminus = df_terminus[['terminus_lon', 'terminus_lat']]
    df_rgi.update(df_terminus)

glaciers_with_no_terminus = df_rgi['terminus_lon'].isna().sum()
print(f'{glaciers_with_no_terminus} of {len(df_rgi)} glaciers have no terminus position '
      f'({100 / len(df_rgi) * glaciers_with_no_terminus:.2f} %)')

1666 of 216502 glaciers have no terminus position (0.77 %)


# Define which coordinates to use for each glacier

In [76]:
# this coordinates are the same as terminus, if terminus does not exist centroid is used
df_rgi['position_lon'] = np.where(df_rgi['terminus_lon'].isna(),
                                  df_rgi['centroid_lon'],
                                  df_rgi['terminus_lon'])
assert df_rgi['position_lon'].isna().sum() == 0

df_rgi['position_lat'] = np.where(df_rgi['terminus_lat'].isna(),
                                  df_rgi['centroid_lat'],
                                  df_rgi['terminus_lat'])
assert df_rgi['position_lat'].isna().sum() == 0

# Save result

In [77]:
base_path = os.getcwd()
# go up until we are in the project base directory
path_to_add = base_path
while path_to_add.split('/')[-1] != 'provide':
    path_to_add = os.path.normpath(os.path.join(path_to_add, '..'))

fp_output = os.path.join(path_to_add, 'general_data_for_aggregation')
filename_output = 'rgi_position_terminus_backdrop_centroid.csv'
df_rgi.to_csv(os.path.join(fp_output, filename_output))